# Defining your own type

Types can be thought as boxes containing fields. Every field has a name and it is highly recoomended to have a type as well (but not required)



In [12]:
type Point{T}
    x::T
    y::T
end

In [32]:
p1 = Point{Int64}(2,3)

Point{Int64}(2,3)

In [36]:
p2 = Point{String}("maria","pepe")

Point{String}("maria","pepe")

In [38]:
p2 = Point("maria","pepe")

Point{String}("maria","pepe")

In [41]:
p2 = Point("maria",3)

LoadError: MethodError: no method matching Point{T}(::String, ::Int64)[0m
Closest candidates are:
  Point{T}{T}(::T, [1m[31m::T[0m) at In[12]:2
  Point{T}{T}(::Any) at sysimg.jl:66[0m

In [53]:
p2 = Point( Point("maria","la"),Point(3,3))

LoadError: MethodError: no method matching Point{T}(::Point{String}, ::Point{Int64})[0m
Closest candidates are:
  Point{T}{T}(::T, [1m[31m::T[0m) at In[12]:2
  Point{T}{T}(::Any) at sysimg.jl:66[0m

In [45]:
p2 = Point( Point(5412,123),Point(3,3))

Point{Point{Int64}}(Point{Int64}(5412,123),Point{Int64}(3,3))

In [34]:
type PointRestricted{T<:Real}
    x::T
    y::T
end

In [35]:
PointRestricted{String}("pepe","pepa")

LoadError: TypeError: PointRestricted: in T, expected T<:Real, got Type{String}

In [49]:
PointRestricted{Int32}(1,45)

PointRestricted{Int32}(1,45)

In [47]:
parse("2+2")

:(2 + 2)

In [48]:
eval(parse("2+2"))

4

In [ ]:
workspace()
type House
    squared_meters
    price
    city
    
    House(squared_meters, price) = new(squared_meters, price, "Unkown")
end


In [ ]:
home = House(60,100000)

#### Parametric types

Parametric types are `Type` objects that take as parameters other `Type` objects or numbers.



In [5]:
type House_p{T<:Real}
    squared_meters::T
    price::T
    city::String
    
    House_p(squared_meters, price) = new(squared_meters, price, "Unkown")
end


In [6]:
home_p = House_p{Float32}(60,100000)

House_p{Float32}(60.0f0,100000.0f0,"Unkown")

In [7]:
typeof(home_p.squared_meters)

Float32

In [11]:
home_p = House_p(60,10,"la")

LoadError: MethodError: no method matching House_p{T<:Real}(::Int64, ::Int64, ::String)[0m
Closest candidates are:
  House_p{T<:Real}{T}(::Any) at sysimg.jl:66[0m

#### Can we use a default parameterized type?

In [ ]:
Array([2])

In [ ]:
type House_2{T<:Real}
    squared_meters::T
    price::T
    city
    
    House_2(squared_meters, price) = new(squared_meters, price, "Unkown")
end


In [ ]:
workspace()
type troll{T}
    weigsht::T
    height::T
end

In [ ]:
a_troll = troll(3,4)

In [ ]:
a_troll

WE can use other types to define a troll

In [ ]:
a_troll = troll(3., 4.)

Nevertheless we defined ty type stating that every filed had to be form the same type. Therefore troll(3,4.) will not work

In [ ]:
a_troll = troll(3,4.)

### Creating inner constructors

We can add a function inside a type used to instanciate the type.

This function can be used for example in order to..

- Put restrictions on the type, such as..
     - not allow negative values 
     - Not allow 'rare' characters

In [ ]:
workspace()

In [ ]:
type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h, i)
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w,h,i)
    end
end

In [ ]:
methodswith(Giant)

In [ ]:
type Family
    name::AbstractString
    members::Array{AbstractString, 1}
    extended::Bool
    # constructor that takes one argument and generates a default
    # for the other two values
    Family(name::AbstractString) = new(name, AbstractString[], false)
    # constructor that takes two arguements and infers the third
    Family(name::AbstractString, members) = new(name, members, length(members) > 3)
end

fam1 = Family("blogs")
println(fam1)

Notice that if a constructor is defined inside a type we must explicitly define a way to instanciate new elements for every possible type T.

In [ ]:
Giant(3.,4.,"low")

Let us do it

In [ ]:
Giant{T}(weight::T, height::T, intelligence::String) = Giant{T}(weight, height, intelligence)

#Giant{T}(weight::T, height::T) = Giant{T}(weight, height ; intelligence="low")

In [ ]:
methodswith(Giant)

In [ ]:
giant = Giant(3., 4., "low")

In [ ]:
giant.height, giant.weight, giant.intelligence

Notice that, sice we have written a restriction on the height of the giant
we cannot instanciate giants with less than 2 meter height

In [ ]:
giant = Giant(3.,4.,"medium")

In [ ]:
giant.height, giant.weight, giant.intelligence

In [ ]:
giant = Giant(3.,1.,"medium")

### Assigning  a default value to a type 

By default we might think that Giants are quite stupid and therefore we might be interested into assigning "stupid" intelligence as default.

Notice that default values in Julia are defined after ```;```

This can be specially handly in order to provide default values for types, for example when there could be 'standard values' for some of the fileds inside a type and we do not want to bother the user selecting field values. 

Using this trick we can allow the user to set values to all the fields the user wants to but at the same time, allow the user **not to** define all possible fields .

In [ ]:
workspace()

In [ ]:
type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h ; intelligence="stupid")
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h ,  intelligence)
    end
    
end

## IMPORTANT TO ADD CONSTRUCTOR FOR EVERY TYPE T !!!!
Giant{T}(weight::T, height::T) = Giant{T}(weight, height)


In [ ]:
giant = Giant(3.,5.)

In [ ]:
giant.weight, giant.height, giant.intelligence

In [ ]:
giant.intelligence = "smart"

In [ ]:
giant.weight, giant.height, giant.intelligence

Notice that we cannot pass now a value for the field intelligence

In [ ]:
rare_giant = Giant(5.,4.,"smart")

Notice that we cannot use this definition since we need to specify how to create a Giant  when we pass the intelligence field.

### Allowing types to have fields with defeault values

https://groups.google.com/forum/#!topic/julia-users/9jM7GIoh7YY

In [ ]:
workspace()

type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h ; intelligence="stupid")
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h, intelligence)
    end
    
    function Giant(w, h, intelligence)
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h, intelligence)
    end
end

## IMPORTANT TO ADD CONSTRUCTOR FOR EVERY TYPE T !!!!
Giant{T}(weight::T, height::T) = Giant{T}(weight, height)
Giant{T}(weight::T, height::T, intelligence::String) = Giant{T}(weight, height, intelligence)

In [ ]:
rare_giant = Giant(5., 4., "smart")

In [ ]:
giant = Giant(3., 5.)

#### Type man


The following example is the type man that contains 3 fields, **```heigh```**,**``` weight```** and **```name```**.

In [ ]:
type man{T}
    height::T
    weight::T
    name::String
end

A type can be instanciated by assigning values to its fields, the fields of a type can be checked with the **```fieldnames```** function

In [ ]:
david = man(183, 80, "David")

In [ ]:
fieldnames(david)

Notice that **you cannot specify a particular value on a field**.

In [ ]:
type girl{T}
    height::T
    weight::T
    name="Julia"
end

Nevertheless you can define a type and a (so called) inner constructor which might specify a particular value of the type.

In [ ]:
type girl2{T}
    height::T
    weight::T
    name::String
end

In [ ]:
girl2(23, 123, "julia")

In [ ]:
typeof(girl2)

In [ ]:
methodswith(girl2)

### Inner constructor

In [ ]:
type girl4{T}
    height::T
    weight::T
    name::String
    
    function girl4(height, weight, name)
        new(23,12,"lala")
    end
end

In [ ]:
girl4(23,123,"julia")

# Making custom print for a defined type

Once defined a type we can import ```Base.show``` and define a ```show``` method for the defined type


In [ ]:
import Base.show

In [ ]:
type Point
       x::Int
       y::Int
end

In [ ]:
Base.show(io::IO, p::Point) = print(io, "Point: x=$(p.x), y=$(p.y)")

In [ ]:
#Base.print(io::IO, p::Point) = string(p)

In [ ]:
Point(4,5)

In [ ]:
print(Point(4,5))

In [ ]:
[Point(2,5), Point(1,1), Point(5,6)]

### Another example

In [ ]:
require("Enum")

using Calendar
   type LogMessage
     stamped::CalendarTime
     msgstat::MsgStatus
     message::String
end
   import Base.show
   show(m::LogMessage) =
   print("$(m.stamped): $(m.msgstat) >> $(m.message)")
   msg = LogMessage(Calendar.now(), WARN, "Be very afraid")
   show(msg)

### The importance of declaring code inside functions

In [ ]:
@time begin
pos = 0
num_steps = 10^4
numwalkers = 10^4
final_square_positions = Int[]

for i in 1:numwalkers
    for j in 1:num_steps
        pos += ifelse( rand() < 0.5, -1, +1)
    end
        push!(final_square_positions, pos^2)
end
    
end

In [ ]:
function fool_function()
    pos = 0
    num_steps = 10^4
    numwalkers = 10^4
    final_square_positions = Int[]

    for i in 1:numwalkers
        for j in 1:num_steps
            pos += ifelse( rand() < 0.5, -1, +1)
        end
            push!(final_square_positions, pos^2)
    end
end

In [ ]:
@time fool_function()